Could not get this to run on wolke.uni-greifswald.de, but in local jupyterhub this works

In [1]:
import tensorflow as tf
import plotly.graph_objects as go
from ipywidgets import widgets
import numpy as np
import math

In [92]:
gammaSlide = widgets.FloatSlider(
    value = 0.2,
    min = 0.01,
    max = 3,
    step = 0.01,
    description = "gamma"
)

nSlide = widgets.IntSlider(
    value = 1000,
    min = 5,
    max = 1000,
    step = 5,
    description = "n"
)

valBoost = widgets.Checkbox(description = 'higher values')
singleBest = widgets.Checkbox(description = 'single highest value')
zscaleSwitch = widgets.Checkbox(description = 'z-normalize')
mscaleSwitch = widgets.Checkbox(description = 'minmax-normalize')
squareSwitch = widgets.Checkbox(description = 'square softmax')
sqrtSwitch = widgets.Checkbox(description = 'sqrt softmax')
doubleSwitch = widgets.Checkbox(description = 'double softmax')
smnormSwitch = widgets.Checkbox(description = 'minmax normalize softmax')
valdivSwitch = widgets.Checkbox(description = 'divide values by softmax')
valmulSwitch = widgets.Checkbox(description = 'multiply values with softmax')

container = widgets.HBox(children=[nSlide, gammaSlide, valBoost, singleBest])
container2 = widgets.HBox(children=[zscaleSwitch, mscaleSwitch, squareSwitch, sqrtSwitch, doubleSwitch, smnormSwitch])
container3 = widgets.HBox(children=[valdivSwitch, valmulSwitch])

In [93]:
values = np.random.gumbel(1, 0.5, nSlide.max)
buf = values[0]
mv = max(values)
mi = np.argmax(values)
values[0] = mv
values[mi] = buf
#print(values)

In [94]:
trace1 = go.Bar(x=list(range(nSlide.value)), y=values[0:nSlide.value], name='values')
trace2 = go.Bar(x=list(range(nSlide.value)), y=tf.nn.softmax(gammaSlide.value * values[0:nSlide.value]).numpy(), name='softmax')
fig = go.FigureWidget(data=[trace1, trace2], 
                      layout=go.Layout(
                          title={'text': "softmax var: "+str(np.var(trace2.y))+"   softmax stdev: "+str(np.std(trace2.y))+"   softmax mean: "+str(np.mean(trace2.y))+"   softmax median: "+str(np.median(trace2.y))}
                      ))

In [95]:
def response(change):
    size = nSlide.value
    gamma = gammaSlide.value
    vals = np.array(values[0:size])
    if valBoost.value:
        vals *= 10
        
    if singleBest.value:
        vals /= 3
        vals[0] *= 3
        
    if zscaleSwitch.value:
        mean = np.mean(vals)
        std = np.std(vals)
        vals -= mean
        vals /= std
        
    if mscaleSwitch.value:
        maxv = np.max(vals)
        minv = np.min(vals)
        vals -= minv
        vals /= (maxv-minv)
        
    smruns = 2 if doubleSwitch.value else 1
    softmax = vals
    for i in range(smruns):
        if doubleSwitch.value and i == 1:
            smmax = np.max(softmax)
            smmin = np.min(softmax)
            softmax -= smmin
            softmax /= (smmax-smmin)
            
        softmax = tf.nn.softmax(gamma*softmax).numpy()

        if squareSwitch.value:
            softmax *= softmax

        if sqrtSwitch.value:
            softmax = np.sqrt(softmax)
    
    if smnormSwitch.value:
        smmax = np.max(softmax)
        smmin = np.min(softmax)
        softmax -= smmin
        softmax /= (smmax-smmin)
        
    if valdivSwitch.value:
        vals /= softmax
        
    if valmulSwitch.value:
        vals *= softmax
    
    with fig.batch_update():
        fig.data[0].x = list(range(size))
        fig.data[1].x = list(range(size))
        fig.data[0].y = vals
        fig.data[1].y = softmax
        fig.layout.title.text = "softmax var: "+str(np.var(softmax))+"   softmax stdev: "+str(np.std(softmax))+"   softmax mean: "+str(np.mean(softmax))+"   softmax median: "+str(np.median(softmax))

In [96]:
gammaSlide.observe(response, names='value')
nSlide.observe(response, names='value')
valBoost.observe(response, names='value')
singleBest.observe(response, names='value')
zscaleSwitch.observe(response, names='value')
mscaleSwitch.observe(response, names='value')
squareSwitch.observe(response, names='value')
sqrtSwitch.observe(response, names='value')
doubleSwitch.observe(response, names='value')
smnormSwitch.observe(response, names='value')
valdivSwitch.observe(response, names='value')
valmulSwitch.observe(response, names='value')

In [97]:
widgets.VBox([container, container2, container3, fig])